    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


In [ ]:
    import IPython, traitlets, jinja2, types, contextlib
    try:
        from .weave import Cell, Document
        from .strings.environment import environment
    except:
        from weave import Cell, Document
        from strings.environment import environment

In [2]:
    ip = get_ipython()

In [16]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        markdown = traitlets.Bool(True) 
        template = traitlets.Bool(True)
        format = traitlets.Bool(True)
        capture = traitlets.Bool(False)
        environment = traitlets.Instance(
            jinja2.Environment, 
            default_value=environment)
        inline = traitlets.Bool(False)
        
        def run_cell(self, raw, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            weave = (ip.autoawait and Cell or Document)(
                    raw, 
                    markdown=getattr(self, 'markdown', True), 
                    template=getattr(self, 'template', True), 
                    inline=getattr(self, 'inline', False), 
                    format=getattr(self, 'format', True),
                    environment=getattr(self, 'environment', environment)
            )
            result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                self, weave.source, store_history=store_history, 
                silent=silent, shell_futures=shell_futures)
            
            IPython.display.display(weave)
                
            return result

In [4]:
    original_methods = {}
    def load_ipython_extension(ip):
        global original_methods
        for method in (PidginShell.run_cell,): 
            object = getattr(ip, method.__name__, None)
            if object:
                original_methods[method.__name__] = original_methods.get(method.__name__, object)
            setattr(ip, method.__name__, types.MethodType(method, ip))

        for trait in (
            'template', 'format', 'environment', 'inline', 'markdown', 'capture'
        ): 
            ip.add_traits(**{trait: getattr(PidginShell,  trait)})
            setattr(ip, trait, getattr(PidginShell,  trait).default_value)
            
        ip.enable_html_pager = True
        
        if controller:
            for control in controller.children:
                name = control.description.split()[0].lower()
                traitlets.link((ip, name), (control, 'value'))

    def unload_ipython_extension(ip):  ip.__dict__.update(original_methods)

In [5]:
    import argparse

In [6]:
    parser = argparse.ArgumentParser(description="""Modify settings for a running pidgin kernel.""")
    parser.add_argument('-m', '--markdown', action='store_false', help='Allow markdown cells as input.')
    parser.add_argument('-c', '--capture', action='store_true', help='Capture the display')
    parser.add_argument('-f', '--format', action='store_false', help='Use normal IPython mode')
    parser.add_argument('-t', '--template', action='store_false', help='Use normal IPython mode')
    parser.add_argument('-i', '--inline', action='store_true', help='Use normal IPython mode');

In [7]:
    try:
        import ipywidgets
        controller = ipywidgets.HBox([
            ipywidgets.Checkbox(description="Markdown"),
            ipywidgets.Checkbox(description="Capture"),
            ipywidgets.Checkbox(description="Format"),
            ipywidgets.Checkbox(description="Template"),
            ipywidgets.Checkbox(description="Inline Expressions"),        
        ])
    except:
        controller = None
        
    def _ipython_display_(): 
        IPython.display.display(controller)

In [8]:
    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())